In [6]:
from keras import layers
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

# 이미지 보여주는 함수

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

# 보간법을 통해서 이미지를 보여준다.     

In [7]:
img_shape = (224, 224, 3)

input_ = layers.Input(shape=img_shape)
x = layers.Conv2D(64, 7, strides=2, padding='same')(input_)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 1, strides=1)(x)
x = layers.Conv2D(192, 3, strides=1, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
x = inception_module(x, o_1=64, r_3=64, o_3=128, r_5=16, o_5=32, pool=32)
x = inception_module(x, o_1=128, r_3=128, o_3=192, r_5=32, o_5=96, pool=64)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
x = inception_module(x, o_1=192, r_3=96, o_3=208, r_5=16, o_5=48, pool=64)
x = inception_module(x, o_1=160, r_3=112, o_3=224, r_5=24, o_5=64, pool=64)
x = inception_module(x, o_1=128, r_3=128, o_3=256, r_5=24, o_5=64, pool=64)
x = inception_module(x, o_1=112, r_3=144, o_3=288, r_5=32, o_5=64, pool=64)
x = inception_module(x, o_1=256, r_3=160, o_3=320, r_5=32, o_5=128, pool=128)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
x = inception_module(x, o_1=256, r_3=160, o_3=320, r_5=32, o_5=128, pool=128)
x = inception_module(x, o_1=384, r_3=192, o_3=384, r_5=48, o_5=128, pool=128)
x = layers.AveragePooling2D(pool_size=(7, 7), strides=1)(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1000)(x)
output = layers.Activation('softmax')(x)

googlenet = Model(input_, output)

In [8]:
print("전체 파라미터 수 : {}".format(sum([arr.flatten().shape[0] for arr in googlenet.get_weights()])))

전체 파라미터 수 : 6956536


In [5]:
def inception_module(x, o_1=64, r_3=64, o_3=128, r_5=16, o_5=32, pool=32):

    x_1 = keras.layers.Conv2D(o_1, 1, padding='same')(x)
    
    x_2 = keras.layers.Conv2D(r_3, 1, padding='same')(x)
    x_2 = keras.layers.Conv2D(o_3, 3, padding='same')(x_2)
    
    x_3 = keras.layers.Conv2D(r_5, 1, padding='same')(x)
    x_3 = keras.layers.Conv2D(o_5, 5, padding='same')(x_3)
    
    x_4 = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=1, padding='same')(x)
    x_4 = keras.layers.Conv2D(pool, 1, padding='same')(x_4)
    
    return keras.layers.concatenate([x_1, x_2, x_3, x_4])